Multiple Linear Regression

In [50]:
from warnings import filterwarnings
filterwarnings('ignore')

In [51]:
path = r"C:\Users\admin\Downloads\Dataset for project.csv"
import pandas as pd
df = pd.read_csv(path,na_values=['','NA'],keep_default_na=False)
df.head()

,market_id,created_at,actual_delivery_time,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_dashers,total_busy_dashers,total_outstanding_orders,estimated_order_place_duration,estimated_store_to_consumer_driving_duration
0,1.0,22:24:17,23:27:16,1845,american,1.0,4,3441,4,557,1239,33.0,14.0,21.0,446,861.0
1,2.0,21:49:25,22:56:29,5477,mexican,2.0,1,1900,1,1400,1400,1.0,2.0,2.0,446,690.0
2,3.0,20:39:28,21:09:09,5477,NaN,1.0,1,1900,1,1900,1900,1.0,0.0,0.0,446,690.0
3,3.0,21:21:45,22:13:00,5477,NaN,1.0,6,6900,5,600,1800,1.0,1.0,2.0,446,289.0
4,3.0,02:40:36,03:20:26,5477,NaN,1.0,3,3900,3,1100,1600,6.0,6.0,9.0,446,650.0


Basic data quality checks

In [52]:
df.shape

(197428, 16)

In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197428 entries, 0 to 197427
Data columns (total 16 columns):
 #   Column                                        Non-Null Count   Dtype  
---  ------                                        --------------   -----  
 0   market_id                                     196441 non-null  float64
 1   created_at                                    197428 non-null  object 
 2   actual_delivery_time                          197421 non-null  object 
 3   store_id                                      197428 non-null  int64  
 4   store_primary_category                        192668 non-null  object 
 5   order_protocol                                196433 non-null  float64
 6   total_items                                   197428 non-null  int64  
 7   subtotal                                      197428 non-null  int64  
 8   num_distinct_items                            197428 non-null  int64  
 9   min_item_price                                19

In [54]:
## Missing values
df.isna().sum()

market_id                                         987
created_at                                          0
actual_delivery_time                                7
store_id                                            0
store_primary_category                           4760
order_protocol                                    995
total_items                                         0
subtotal                                            0
num_distinct_items                                  0
min_item_price                                      0
max_item_price                                      0
total_onshift_dashers                           16262
total_busy_dashers                              16262
total_outstanding_orders                        16262
estimated_order_place_duration                      0
estimated_store_to_consumer_driving_duration      526
dtype: int64

In [55]:
df.duplicated().sum()

np.int64(0)

In [56]:
df.drop_duplicates()

,market_id,created_at,actual_delivery_time,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_dashers,total_busy_dashers,total_outstanding_orders,estimated_order_place_duration,estimated_store_to_consumer_driving_duration
0,1.0,22:24:17,23:27:16,1845,american,1.0,4,3441,4,557,1239,33.0,14.0,21.0,446,861.0
1,2.0,21:49:25,22:56:29,5477,mexican,2.0,1,1900,1,1400,1400,1.0,2.0,2.0,446,690.0
2,3.0,20:39:28,21:09:09,5477,NaN,1.0,1,1900,1,1900,1900,1.0,0.0,0.0,446,690.0
3,3.0,21:21:45,22:13:00,5477,NaN,1.0,6,6900,5,600,1800,1.0,1.0,2.0,446,289.0
4,3.0,02:40:36,03:20:26,5477,NaN,1.0,3,3900,3,1100,1600,6.0,6.0,9.0,446,650.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197423,1.0,00:19:41,01:24:48,2956,fast,4.0,3,1389,3,345,649,17.0,17.0,23.0,251,331.0
197424,1.0,00:01:59,00:58:22,2956,fast,4.0,6,3010,4,405,825,12.0,11.0,14.0,251,915.0
197425,1.0,04:46:08,05:36:16,2956,fast,4.0,5,1836,3,300,399,39.0,41.0,40.0,251,795.0
197426,1.0,18:18:15,19:23:22,3630,sandwich,1.0,1,1175,1,535,535,7.0,7.0,12.0,446,384.0


Convert the time columns to datetime

In [57]:
df ['created_at']=pd.to_datetime(df ['created_at'])
df ['actual_delivery_time']=pd.to_datetime(df ['actual_delivery_time'])

Extract time based features


Create a new column for actual time taken for delivery taken in seconds 

In [58]:
df['delivery_duration_seconds']=(df['actual_delivery_time']-df['created_at']).dt.total_seconds()

extract date and time components

In [59]:
df ['created_date']= df ['created_at'].dt.date
df ['created_time']= df ['created_at'].dt.time
df ['delivery_date']=df ['actual_delivery_time'].dt.date
df ['delivery_time']=df ['actual_delivery_time'].dt.time

There are  missing values but no  an duplicated rows in the data
Separate X and Y features
 
X : REMAINING df FEATURES except delivery_duration_seconds
Y : delivery_duration_seconds

In [60]:
X = df.drop(columns=['delivery_duration_seconds','created_at','actual_delivery_time'])
Y = df[['delivery_duration_seconds']]

In [61]:
X.dtypes

market_id                                       float64
store_id                                          int64
store_primary_category                           object
order_protocol                                  float64
total_items                                       int64
subtotal                                          int64
num_distinct_items                                int64
min_item_price                                    int64
max_item_price                                    int64
total_onshift_dashers                           float64
total_busy_dashers                              float64
total_outstanding_orders                        float64
estimated_order_place_duration                    int64
estimated_store_to_consumer_driving_duration    float64
created_date                                     object
created_time                                     object
delivery_date                                    object
delivery_time                                   

Separate categorical and continuous columns

In [62]:
cat = list(X.columns[X.dtypes=='object'])
con = list(X.columns[X.dtypes!='object'])
print(cat,con,sep='\n')

['store_primary_category', 'created_date', 'created_time', 'delivery_date', 'delivery_time']
['market_id', 'store_id', 'order_protocol', 'total_items', 'subtotal', 'num_distinct_items', 'min_item_price', 'max_item_price', 'total_onshift_dashers', 'total_busy_dashers', 'total_outstanding_orders', 'estimated_order_place_duration', 'estimated_store_to_consumer_driving_duration']


Data preprocessing and Data Cleaning: 1.One hot encoding=> categorical data 2.Standard scale=> continuous data

In [63]:
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,OrdinalEncoder
from sklearn.compose import ColumnTransformer

In [64]:
cat_pipe = make_pipeline(
    SimpleImputer(strategy='most_frequent'),
OrdinalEncoder() # feature scaling
)


In [65]:
con_pipe = make_pipeline(
    SimpleImputer(strategy="mean"), # data cleaning
    StandardScaler()
)

In [66]:
pre= ColumnTransformer([
    ("cat",cat_pipe,cat),
    ("con",con_pipe,con)
]).set_output(transform='pandas')

In [67]:
mean_value=Y.mean()
Y.fillna(mean_value,inplace = True)

In [68]:
pre

,transformers,"[('cat', ...), ('con', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,'deprecated'
,missing_values,nan
,strategy,'most_frequent'
,fill_value,None


In [69]:
X_pre= pre.fit_transform (X)

Train Test Split
rate at which you can divide:

100%

60 % training 40% testing

70% training 30% testing

80% training 20% testing
random_state = this is generally used for reproducing model result

In [70]:
from sklearn.model_selection import train_test_split

In [71]:
xtrain,xtest,ytrain,ytest = train_test_split(X_pre,Y,train_size=0.7,test_size=0.3,random_state=21)

In [72]:
xtrain.head()

,cat__store_primary_category,cat__created_date,cat__created_time,cat__delivery_date,cat__delivery_time,con__market_id,con__store_id,con__order_protocol,con__total_items,con__subtotal,con__num_distinct_items,con__min_item_price,con__max_item_price,con__total_onshift_dashers,con__total_busy_dashers,con__total_outstanding_orders,con__estimated_order_place_duration,con__estimated_store_to_consumer_driving_duration
191851,69.0,0.0,12186.0,0.0,14037.0,-0.643443,0.194542,-1.254924,-0.073650,0.220872,0.201937,0.399936,-0.115665,0.761679,0.657944,7.126387e-01,1.524747,0.491377
128175,51.0,0.0,45781.0,0.0,1584.0,0.014000,-0.633804,0.078434,-0.823686,-0.860260,-1.024867,0.265846,-0.599181,-0.961720,-1.095685,-1.011970e+00,-0.638568,-0.654428
150540,59.0,0.0,4825.0,0.0,6748.0,0.671442,0.999512,-0.588245,1.051404,0.101843,0.815340,-1.011840,0.555884,0.005802,-0.024023,-9.924345e-04,-0.638568,0.249434
80906,73.0,0.0,32898.0,0.0,35012.0,0.671442,1.359874,-0.588245,-0.448668,-0.407183,-0.411465,0.495715,-0.294745,-0.357019,-0.348769,-1.595771e-01,-0.638568,-0.033593
137931,46.0,0.0,21205.0,0.0,23006.0,1.986326,1.333577,-0.588245,0.301368,0.741965,0.815340,-0.260936,0.958814,0.000000,0.000000,-1.408515e-16,-0.638568,1.869995


In [73]:
xtest.head()

,cat__store_primary_category,cat__created_date,cat__created_time,cat__delivery_date,cat__delivery_time,con__market_id,con__store_id,con__order_protocol,con__total_items,con__subtotal,con__num_distinct_items,con__min_item_price,con__max_item_price,con__total_onshift_dashers,con__total_busy_dashers,con__total_outstanding_orders,con__estimated_order_place_duration,con__estimated_store_to_consumer_driving_duration
67513,4.0,0.0,29282.0,0.0,31242.0,1.328884,-1.023384,-0.588245,-0.073650,-0.113177,0.201937,-0.167073,-0.669022,-0.931485,-1.160634,-0.833562,-0.638568,-0.234452
178218,4.0,0.0,33552.0,0.0,35349.0,1.328884,-0.052842,-1.254924,-0.823686,-0.587646,-1.024867,0.706428,-0.121038,-0.689604,-0.835888,-0.912854,1.524747,-1.170268
61453,60.0,0.0,5311.0,0.0,10262.0,-0.643443,-1.271254,1.411791,-0.073650,0.914749,0.201937,1.079964,0.788688,1.638497,1.242487,1.644324,-0.638568,1.107647
22816,39.0,0.0,9310.0,0.0,12833.0,-0.643443,-1.495750,1.411791,1.051404,1.764954,0.201937,-0.165158,0.609608,1.729202,1.274961,1.386624,-0.638568,0.395513
39264,40.0,0.0,5768.0,0.0,8989.0,-1.300885,1.619919,-1.254924,-0.448668,1.984362,-0.411465,3.474429,4.728446,-1.264071,-1.258058,-1.091262,1.524747,-1.302652


In [74]:
xtrain.index

Index([191851, 128175, 150540,  80906, 137931, 128172,  17680, 186496,  47446,
        44181,
       ...
       132718, 110794,  64241, 176738,  42104,  63856,  81968, 140036,  70863,
        80841],
      dtype='int64', length=138199)

In [75]:
ytrain.head()

,delivery_duration_seconds
191851,2128.0
128175,-84301.0
150540,2200.0
80906,2298.0
137931,1600.0


In [76]:
ytest.head()

,delivery_duration_seconds
67513,2064.0
178218,1976.0
61453,5230.0
22816,3801.0
39264,3499.0


In [77]:

model.fit(xtrain,ytrain)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [78]:
from sklearn.linear_model import LinearRegression
model=LinearRegression()
model.fit (X_pre,Y)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [79]:
model.intercept_

array([-2041.27093326])

In [80]:
ypreds = model.predict(X_pre)
ypreds

array([[4184.18363152],
       [4705.15701234],
       [ 752.14740735],
       ...,
       [3144.03909913],
       [4379.0545598 ],
       [2050.31722353]], shape=(197428, 1))

Evaluate the model
Mean Sqaured Error
Mean Absolute Error
RMSE
R2 score- Coeff of determination

In [81]:
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score

In [82]:
mse = mean_squared_error(Y,ypreds)
mae = mean_absolute_error(Y,ypreds)
rmse = mse**(1/2)
r2 = r2_score(Y,ypreds)

print(f"MSE: {mse:.2f}")
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R2 score: {r2*100:.2f}%")

MSE: 699607.75
MAE: 567.63
RMSE: 836.43
R2 score: 99.68%


Model can be used for final predictions since the r2 score is greater than 80%
Out of sample predictions